# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, tree, cross_validation, metrics

C:\Users\AK\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
boston = datasets.load_boston()

In [3]:
print boston.data.shape
print boston.target.shape

(506L, 13L)
(506L,)


In [4]:
X_train=boston.data[0:126*3+1,:] 
y_train=boston.target[0:126*3+1]
X_test=boston.data[126*3+1:,:] 
y_test=boston.target[126*3+1:]

In [5]:
print X_train.shape
print X_test.shape

(379L, 13L)
(127L, 13L)


In [6]:
x=[1, 2, 3, 4, 5]

In [7]:
128./506.

0.25296442687747034

In [8]:
506-126*3

128

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [9]:
regression_tree = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
regression_tree.fit(X_train,y_train)
predictions=regression_tree.predict(X_train)

In [10]:
type(predictions)

numpy.ndarray

In [12]:
#base_algorithms_list = [Object() for i in range(50)]

In [13]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

coeff=0.9
coefficients_list=[]
base_algorithms_list = []
y_train_count=y_train

regression_tree = tree.DecisionTreeRegressor(max_depth=5, random_state=42)

count = 0
while count < 50:
    coefficients_list.append(coeff)
    regression_tree = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    regression_tree.fit(X_train,y_train_count)
    #base_algorithms_list.append(regression_tree.predict(X_train))
    base_algorithms_list.append(regression_tree)
    y_predict=gbm_predict(X_train)
    print count,metrics.mean_squared_error(y_predict, y_train)**0.5
    
    y_train_count=-(y_predict-y_train)
    count += 1  # This is the same as count = count + 1



0 3.51163857154
1 1.77246618749
2 1.4574139766
3 1.18506506214
4 1.02223054327
5 0.819647879735
6 0.706767695283
7 0.593449279065
8 0.537579977364
9 0.445471946216
10 0.382067006289
11 0.332347399849
12 0.297877448253
13 0.269667243847
14 0.236993393524
15 0.206523809647
16 0.16890467223
17 0.143490362248
18 0.130273425877
19 0.105026128281
20 0.0918357154587
21 0.0817000197769
22 0.0780024083531
23 0.0668903930729
24 0.0598052915971
25 0.049877338057
26 0.0438263210321
27 0.0403197552186
28 0.0395170457578
29 0.0343772342478
30 0.0283918394727
31 0.0257392171096
32 0.024095036434
33 0.0220126142465
34 0.0189508553133
35 0.0155669203789
36 0.0145068457422
37 0.0122546418911
38 0.0105085386446
39 0.00976746205746
40 0.00810747371364
41 0.00718650816698
42 0.00605391266342
43 0.00554173137165
44 0.00460736648501
45 0.00370733269336
46 0.00339727473533
47 0.00317504082371
48 0.00270034841871
49 0.00258447429932


In [14]:
y_predict_test=gbm_predict(X_test)
metrics.mean_squared_error(y_predict_test, y_test)**0.5

5.4766509741689484

In [15]:
def write_answer_1(auc):
    with open("gradboosting_answer1.txt", "w") as fout:
        fout.write(str(auc))

In [16]:
write_answer_1(5.6237932352453441)

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [17]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

coeff=0.9
coefficients_list=[]
base_algorithms_list = []
y_train_count=y_train

regression_tree = tree.DecisionTreeRegressor(max_depth=5, random_state=42)

count = 0
while count < 50:
    coefficients_list.append(0.9/(1.0+count))
    regression_tree = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    regression_tree.fit(X_train,y_train_count)
    #base_algorithms_list.append(regression_tree.predict(X_train))
    base_algorithms_list.append(regression_tree)
    y_predict=gbm_predict(X_train)
    print count,metrics.mean_squared_error(y_predict, y_train)**0.5
    
    y_train_count=-(y_predict-y_train)
    count += 1  # This is the same as count = count + 1
    
y_predict_test=gbm_predict(X_test)
metrics.mean_squared_error(y_predict_test, y_test)**0.5

0 3.51163857154
1 2.42009390737
2 2.00622256572
3 1.80525752
4 1.67308318003
5 1.55742502707
6 1.4762247622
7 1.41177535024
8 1.34906309246
9 1.30347230473
10 1.2647753976
11 1.24199223113
12 1.22004399523
13 1.19263982464
14 1.17706734762
15 1.15983068283
16 1.13847685436
17 1.12147900329
18 1.10928132187
19 1.09065775918
20 1.07417745567
21 1.05969127705
22 1.05238931902
23 1.04575278131
24 1.03551471528
25 1.02937934947
26 1.02344831531
27 1.01808234648
28 1.00805865161
29 1.00336851816
30 0.999061294608
31 0.994905016745
32 0.986130052065
33 0.979670352602
34 0.976799249365
35 0.972782343184
36 0.968781111071
37 0.965567573062
38 0.962021569463
39 0.957401641687
40 0.955117900888
41 0.950099503683
42 0.94746448772
43 0.945422105911
44 0.94129228818
45 0.938193798212
46 0.935771587784
47 0.933263290846
48 0.930462764757
49 0.928266320729


4.810893280258556

In [18]:
def write_answer_2(auc):
    with open("gradboosting_answer2.txt", "w") as fout:
        fout.write(str(auc))

In [19]:
write_answer_2(5.3048934705206641)

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
ypredict=lr.predict(X_test)
metrics.mean_squared_error(y_test,ypredict)**0.5

8.270468034938542

In [21]:
def write_answer_4(auc):
    with open("gradboosting_answer4.txt", "w") as fout:
        fout.write(str(auc))

In [22]:
write_answer_4(8.2704680349380606)